In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/penyisihan-hology-7-data-mining-competition/sample_submission.csv
/kaggle/input/penyisihan-hology-7-data-mining-competition/train.csv
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/1075.jpg
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/820.jpg
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/1024.jpg
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/785.jpg
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/792.jpg
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/1009.jpg
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/915.jpg
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/815.jpg
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/817.jpg
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/1055.jpg
/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test/795.jpg
/kaggle/inpu

<h2>5. Convolutional Neural Network</h2>

Convolutional Neural Network (CNN) adalah jenis arsitektur jaringan saraf tiruan yang terutama digunakan untuk memproses data yang memiliki grid-like topology, seperti gambar. CNN sangat efektif dalam tugas-tugas seperti pengenalan gambar, analisis video, dan pemrosesan sinyal suara. CNN terdiri atas beberapa lapisan utama:

**1. Convolutional Layer**<br>
Lapisan ini merupakan inti dari CNN. Pada lapisan ini, sebuah filter atau kernel (matriks kecil) diterapkan pada data input (misalnya gambar) untuk mengekstrak fitur-fitur penting, seperti tekstur, tepi, atau pola. Proses dalam lapisan ini menghasilkan peta fitur (feature map) yang menunjukan lokasi dan intensitas dari fitur-fitur yang terdeteksi.

Secara visual, berikut merupakan ilustrasi dari ekstraksi fitur oleh sebuah lapisan convolutional.
<center><img src="https://analyticsindiamag.com/wp-content/uploads/2018/01/conv-full-layer.gif" width="500" height="500"></center>
<a href="https://www.youtube.com/watch?v=3JQ3hYko51Y8" target="_blank">Lihat animasinya disini</a><br>

**2. Activation Layer**<br>
Setelah peta fitur didapatkan dari proses konvolusi, hasilnya biasanya akan diteruskan ke fungsi aktivasi. Fungsi aktivasi memiliki peran dalam memberikan non-linearitas dari peta fitur agar model dapat memproses fitur kompleks. Apabila fungsi aktivasi tidak ada, maka neural network kita hanya akan menjadi sebuah model berbasis regresi linear.

**3. Pooling Layer**<br>
Pooling berfungsi untuk mengurangi dimensi dari peta fitur dan memudahkan pemrosesan lebih lanjut. Proses ini dilakukan dengan mengambil nilai maksimum (max pooling) atau rata-rata (average pooling) dari area tertentu dalam peta fitur. Pooling membantu dalam membuat model lebih tahan terhadap translasi dan distorsi pada input.

**4. Fully Connected Layer (FC Layer)**<br>
Setelah beberapa lapisan convolutional dan pooling, keluaran dari lapisan-lapisan tersebut biasanya dipipihkan menjadi vektor 1D dan diteruskan ke satu atau lebih lapisan fully connected. Lapisan ini menghubungkan setiap neuron di lapisan satu ke neuron di lapisan berikutnya, yang digunakan untuk klasifikasi akhir dalam tugas pengenalan gambar.

<h2>6. Studi Kasus: Klasifikasi Tumor Otak Menggunakan Computer Vision</h2>
Sekarang mari kita buat neural network kita secara langsung dengan PyTorch. Ikuti instruksi berikut sebelum memulai notebook.

<center><img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*vZtP98UkBRjxzrvqywJZuw.png" width="500" height="500"></center>

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm

print(f"Cek GPU:", "Terhubung" if torch.cuda.is_available() else "Tidak ada GPU")

Cek GPU: Terhubung


In [14]:
# class SimpleCNN(nn.Module):
#     def __init__(self, num_classes_jenis=2, num_classes_warna=5):
#         super(SimpleCNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
#         # Fully connected layers
#         self.fc1 = nn.Linear(50*50*32, 128)
        
#         # Output layers for each classification task
#         self.fc2_jenis = nn.Linear(128, num_classes_jenis)  # For `jenis`
#         self.fc2_warna = nn.Linear(128, num_classes_warna)  # For `warna`
        
#         self.relu = nn.ReLU()
#         self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = self.conv2(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         x = self.relu(x)
        
#         # Two outputs: one for `jenis` and one for `warna`
#         output_jenis = self.fc2_jenis(x)
#         output_warna = self.fc2_warna(x)
        
#         return output_jenis, output_warna


In [19]:
import torch
import torch.nn as nn

class ComplexCNN(nn.Module):
    def __init__(self, num_classes_jenis=2, num_classes_warna=5):
        super(ComplexCNN, self).__init__()
        
        # Initial convolution with larger kernel for better feature capture
        self.conv_initial = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # First parallel path - focus on fine details (smaller kernels)
        self.path1 = nn.Sequential(
            self._make_conv_block(64, 128, kernel_size=3),
            self._make_conv_block(128, 256, kernel_size=3),
            self._make_conv_block(256, 512, kernel_size=3)
        )
        
        # Second parallel path - focus on broader features (larger kernels)
        self.path2 = nn.Sequential(
            self._make_conv_block(64, 128, kernel_size=5),
            self._make_conv_block(128, 256, kernel_size=5),
            self._make_conv_block(256, 512, kernel_size=5)
        )
        
        # Third parallel path - focus on global structure
        self.path3 = nn.Sequential(
            self._make_conv_block(64, 128, kernel_size=7),
            self._make_conv_block(128, 256, kernel_size=7),
            self._make_conv_block(256, 512, kernel_size=7)
        )
        
        # Squeeze and Excitation blocks for each path
        self.se1 = SEBlock(512)
        self.se2 = SEBlock(512)
        self.se3 = SEBlock(512)
        
        # Attention mechanism for feature fusion
        self.attention = SpatialAttention()
        
        # Calculate the correct input size for the classifiers
        self.feature_size = 7 * 7 * (512 * 3)  # 75,264
        
        # Different fully connected paths for jenis and warna
        self.jenis_classifier = nn.Sequential(
            nn.Linear(75264, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes_jenis)
        )
        
        self.warna_classifier = nn.Sequential(
            nn.Linear(75264, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes_warna)
        )
    
    def _make_conv_block(self, in_channels, out_channels, kernel_size):
        padding = kernel_size // 2
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, 
                     stride=1, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, 
                     stride=1, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
    
    def forward(self, x):
        # Initial convolution
        x = self.conv_initial(x)
        print(f"After initial conv: {x.shape}")
        
        # Process through parallel paths
        x1 = self.path1(x)
        x2 = self.path2(x)
        x3 = self.path3(x)
        print(f"After parallel paths: {x1.shape}, {x2.shape}, {x3.shape}")
        
        # Apply SE blocks
        x1 = self.se1(x1)
        x2 = self.se2(x2)
        x3 = self.se3(x3)
        
        # Apply attention
        x1 = self.attention(x1) * x1
        x2 = self.attention(x2) * x2
        x3 = self.attention(x3) * x3
        
        # Concatenate features
        x = torch.cat((x1, x2, x3), dim=1)
        print(f"After concatenation: {x.shape}")
        
        # Flatten
        x = x.view(x.size(0), -1)
        print(f"After flattening: {x.shape}")
        
        # Separate classification paths
        jenis_out = self.jenis_classifier(x)
        warna_out = self.warna_classifier(x)
        
        return jenis_out, warna_out

# Squeeze and Excitation Block
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

# Spatial Attention Module
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size//2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv(x)
        return self.sigmoid(x)

In [20]:
sample_input = torch.randn(1, 3, 200, 200)
model = ComplexCNN()
jenis_out, warna_out = model(sample_input)
print(f"Jenis output shape: {jenis_out.shape}")
print(f"Warna output shape: {warna_out.shape}")

After initial conv: torch.Size([1, 64, 50, 50])
After parallel paths: torch.Size([1, 512, 6, 6]), torch.Size([1, 512, 6, 6]), torch.Size([1, 512, 6, 6])
After concatenation: torch.Size([1, 1536, 6, 6])
After flattening: torch.Size([1, 55296])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x55296 and 75264x1024)

In [ ]:
# data_dir = "./data/" # Kalau di local
data_dir = "/kaggle/input/penyisihan-hology-7-data-mining-competition/" # Di google colab
metadata = pd.read_csv(data_dir + "train.csv")

In [ ]:
metadata.head()

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, image_dir, metadata_df, transform=None):
        self.image_dir = image_dir
        self.metadata_df = metadata_df
        self.transform = transform
        self.valid_extensions = ['.jpg', '.jpeg', '.png']  # Add more extensions if needed
        
        # Ensure metadata is sorted by id to match with images
        self.metadata_df = self.metadata_df.sort_values(by='id')
        
    def __len__(self):
        return len(self.metadata_df)
    
    def find_image_file(self, image_id):
        # Try each extension
        for ext in self.valid_extensions:
            img_path = os.path.join(self.image_dir, f"{image_id}{ext}")
            if os.path.exists(img_path):
                return img_path
        raise FileNotFoundError(f"No image file found for ID {image_id} with extensions {self.valid_extensions}")
    
    def __getitem__(self, idx):
        # Get metadata for this index
        row = self.metadata_df.iloc[idx]
        
        # Find the correct image file
        img_path = self.find_image_file(row['id'])
        
        # Load image
        image = Image.open(img_path).convert('RGB')
        
        # Apply transforms if any
        if self.transform:
            image = self.transform(image)
            
        # Get labels from metadata
        jenis = torch.tensor(row['jenis'], dtype=torch.long)
        warna = torch.tensor(row['warna'], dtype=torch.long)
        
        return {
            'image': image,
            'jenis': jenis,
            'warna': warna
        }

# Create dataset
dataset = CustomDataset(
    image_dir=data_dir + "train/train",
    metadata_df=metadata,
    transform=transform
)

In [ ]:
# Split data ke train, validasi, dan tes. Proporsi data train 70%, validasi 15%, dan tes 15%
train_size = int(0.7 * len(dataset))
eval_size = len(dataset) - train_size
val_size = int(0.5 * eval_size)
test_size = eval_size - val_size

train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])
val_dataset, test_dataset = random_split(eval_dataset, [val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
# Model, loss function, optimizer and scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)  # Adjust num_classes based on your needs
criterion = nn.CrossEntropyLoss()  # Negative log likelihood loss for classification
optimizer = optim.Adam(model.parameters(), lr=3e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, verbose=1)

# Initialize early stopping
early_stopping = EarlyStopping(patience=10, verbose=True)

# Training and validation loop
num_epochs = 60
best_val_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_jenis = 0
    correct_warna = 0
    total = 0
    
    for batch in tqdm(train_loader):
        # Unpack the dictionary
        images = batch['image'].to(device)
        jenis_labels = batch['jenis'].to(device)
        warna_labels = batch['warna'].to(device)
        
        optimizer.zero_grad()
        
        # Forward pass - now you get two outputs: for `jenis` and `warna`
        outputs_jenis, outputs_warna = model(images)
        
        # Calculate loss for both `jenis` and `warna`
        loss_jenis = criterion(outputs_jenis, jenis_labels)
        loss_warna = criterion(outputs_warna, warna_labels)
        
        # Combine the two losses
        loss = loss_jenis + loss_warna
        
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
        # Calculate accuracy for both `jenis` and `warna`
        _, predicted_jenis = torch.max(outputs_jenis, 1)
        _, predicted_warna = torch.max(outputs_warna, 1)
        
        total += jenis_labels.size(0)
        correct_jenis += (predicted_jenis == jenis_labels).sum().item()
        correct_warna += (predicted_warna == warna_labels).sum().item()
    
    # Calculate average loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc_jenis = correct_jenis / total
    epoch_acc_warna = correct_warna / total
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc (Jenis): {epoch_acc_jenis:.4f}, Train Acc (Warna): {epoch_acc_warna:.4f}')
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_jenis = 0
    correct_warna = 0
    total = 0
    
    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            jenis_labels = batch['jenis'].to(device)
            warna_labels = batch['warna'].to(device)
            
            outputs_jenis, outputs_warna = model(images)
            
            # Calculate loss for both `jenis` and `warna`
            loss_jenis = criterion(outputs_jenis, jenis_labels)
            loss_warna = criterion(outputs_warna, warna_labels)
            loss = loss_jenis + loss_warna
            
            val_loss += loss.item() * images.size(0)
            
            # Calculate accuracy for both `jenis` and `warna`
            _, predicted_jenis = torch.max(outputs_jenis, 1)
            _, predicted_warna = torch.max(outputs_warna, 1)
            
            total += jenis_labels.size(0)
            correct_jenis += (predicted_jenis == jenis_labels).sum().item()
            correct_warna += (predicted_warna == warna_labels).sum().item()
    
    # Calculate average validation loss and accuracy
    val_loss /= len(val_loader.dataset)
    val_acc_jenis = correct_jenis / total
    val_acc_warna = correct_warna / total
    print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Acc (Jenis): {val_acc_jenis:.4f}, Val Acc (Warna): {val_acc_warna:.4f}')
    
    # Save best model based on validation accuracy (for `jenis`)
    if val_acc_jenis > best_val_accuracy:
        best_val_accuracy = val_acc_jenis
        torch.save(model.state_dict(), 'best_model.pth')
    
    # Early stopping
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break
    
    # Step the scheduler
    scheduler.step(val_loss)

print('Finished Training')


In [ ]:
val_loss_jenis = 0.0
val_loss_warna = 0.0
correct_jenis = 0
correct_warna = 0
total = 0

all_jenis_labels = []
all_warna_labels = []
all_jenis_predictions = []
all_warna_predictions = []

model.eval()  # Set model to evaluation mode
with torch.no_grad():
    for batch in test_loader:
        images = batch['image'].to(device)
        jenis_labels = batch['jenis'].to(device)
        warna_labels = batch['warna'].to(device)
        
        # Model forward pass (assuming it outputs two sets of predictions)
        outputs_jenis, outputs_warna = model(images)
        
        # Calculate loss for both `jenis` and `warna`
        loss_jenis = criterion(outputs_jenis, jenis_labels)
        loss_warna = criterion(outputs_warna, warna_labels)
        
        # Accumulate losses
        val_loss_jenis += loss_jenis.item() * images.size(0)
        val_loss_warna += loss_warna.item() * images.size(0)
        
        # Get predictions for both
        _, predicted_jenis = torch.max(outputs_jenis, 1)
        _, predicted_warna = torch.max(outputs_warna, 1)
        
        # Accumulate total count and correct predictions
        total += jenis_labels.size(0)
        correct_jenis += (predicted_jenis == jenis_labels).sum().item()
        correct_warna += (predicted_warna == warna_labels).sum().item()

        # Store labels and predictions for confusion matrix and classification report
        all_jenis_labels.extend(jenis_labels.cpu().numpy())
        all_warna_labels.extend(warna_labels.cpu().numpy())
        all_jenis_predictions.extend(predicted_jenis.cpu().numpy())
        all_warna_predictions.extend(predicted_warna.cpu().numpy())

# Calculate accuracy
test_acc_jenis = correct_jenis / total
test_acc_warna = correct_warna / total
average_val_loss_jenis = val_loss_jenis / total
average_val_loss_warna = val_loss_warna / total

# Print accuracy
print(f'Test Accuracy (Jenis): {test_acc_jenis:.4f}')
print(f'Average Validation Loss (Jenis): {average_val_loss_jenis:.4f}')
print(f'Test Accuracy (Warna): {test_acc_warna:.4f}')
print(f'Average Validation Loss (Warna): {average_val_loss_warna:.4f}')

# Calculate and print confusion matrix and classification report for `jenis`
conf_matrix_jenis = confusion_matrix(all_jenis_labels, all_jenis_predictions)
class_report_jenis = classification_report(all_jenis_labels, all_jenis_predictions)

print('Confusion Matrix (Jenis):')
print(conf_matrix_jenis)
print('\nClassification Report (Jenis):')
print(class_report_jenis)

# Calculate and print confusion matrix and classification report for `warna`
conf_matrix_warna = confusion_matrix(all_warna_labels, all_warna_predictions)
class_report_warna = classification_report(all_warna_labels, all_warna_predictions)

print('Confusion Matrix (Warna):')
print(conf_matrix_warna)
print('\nClassification Report (Warna):')
print(class_report_warna)


In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import os

class InferenceDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.valid_extensions = ['.jpg', '.jpeg', '.png']  # Add more extensions if needed
        
        # Collect all valid image paths
        self.image_files = [f for f in os.listdir(image_dir) if os.path.splitext(f)[1].lower() in self.valid_extensions]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Get image filename
        img_filename = self.image_files[idx]
        
        # Construct full path to image
        img_path = os.path.join(self.image_dir, img_filename)
        
        # Load image
        image = Image.open(img_path).convert('RGB')
        
        # Apply transforms if any
        if self.transform:
            image = self.transform(image)
        
        # Return image and filename (for reference)
        return {
            'image': image,
            'filename': img_filename
        }

In [ ]:
# Create dataset for inference
inference_dataset = InferenceDataset(
    image_dir=data_dir + "test/test",  # Path to the inference images
    transform=transform
)

In [ ]:
inference_loader = DataLoader(inference_dataset, batch_size=32, shuffle=True)

In [ ]:
res_jenis_predictions = []
res_warna_predictions = []
res_file_names = []

model.eval()  # Set model to evaluation mode
with torch.no_grad():
    for batch in inference_loader:
        images = batch['image'].to(device)
        filenames = [name.split(".")[0] for name in batch['filename']]
        
        # Model forward pass (assuming it outputs two sets of predictions)
        outputs_jenis, outputs_warna = model(images)
        
        # Get predictions for both
        _, predicted_jenis = torch.max(outputs_jenis, 1)
        _, predicted_warna = torch.max(outputs_warna, 1)
        
        res_jenis_predictions.extend(predicted_jenis.cpu().numpy())
        res_warna_predictions.extend(predicted_warna.cpu().numpy())
        res_file_names.extend(filenames)

In [ ]:
import pandas as pd

# Create a DataFrame to hold test predictions
df_predictions = pd.DataFrame({
    'id': res_file_names,
    'jenis': res_jenis_predictions,
    'warna': res_warna_predictions
})

# Save the DataFrame to a CSV file
df_predictions.to_csv('test_predictions.csv', index=False)

print("Predictions saved to 'test_predictions.csv'")

y